In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
  if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/InstructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

<ipython-input-3-925b459844e6>:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  use_mps = True if torch.has_mps else False


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [ ]:
task_name = 'joint_task'
experiment_name = 'try_joint_task'
model_checkpoint = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined'
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")

In [ ]:
import json
import pandas as pd
import ast

# Load the data
train_file_path = './Dataset/labeled_data.csv'  ### <-- Update with your data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data into a pandas DataFrame
try:
    df = pd.read_csv(train_file_path, sep=';')
except FileNotFoundError:
    print(f"Error: File not found at {train_file_path}")
    # Handle the error appropriately, e.g., exit the script or use a default DataFrame
    exit()

df = df.dropna(subset=['aspect_term_sentiment'])


# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))

Training set size: 1524
Validation set size: 382


In [ ]:
id_tr_df = train_df
id_val_df = val_df

In [ ]:
print(id_tr_df.columns)

Index(['id', 'kode_produk', 'review', 'aspect_term_sentiment'], dtype='object')


In [ ]:
print(id_val_df.columns)

Index(['id', 'kode_produk', 'review', 'aspect_term_sentiment'], dtype='object')


In [ ]:
# Clean column names
id_tr_df.columns = id_tr_df.columns.str.strip()
id_val_df.columns = id_val_df.columns.str.strip()


# Ensure 'aspectTerms' is in the correct format (list/dictionary)
def convert_to_list(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []

id_tr_df['aspect_term_sentiment'] = id_tr_df['aspect_term_sentiment'].apply(convert_to_list)
id_val_df['aspect_term_sentiment'] = id_val_df['aspect_term_sentiment'].apply(convert_to_list)

In [ ]:
print(id_tr_df['aspect_term_sentiment'].head())

141           [{'term': 'aroma', 'polarity': 'negative'}]
1302        [{'term': 'tekstur', 'polarity': 'positive'}]
1124    [{'term': 'tekstur', 'polarity': 'positive'}, ...
1780        [{'term': 'lainnya', 'polarity': 'positive'}]
339          [{'term': 'lainnya', 'polarity': 'neutral'}]
Name: aspect_term_sentiment, dtype: object


In [ ]:
print(id_val_df['aspect_term_sentiment'].head())

1226        [{'term': 'lainnya', 'polarity': 'positive'}]
111     [{'term': 'whitecast', 'polarity': 'positive'}...
1839        [{'term': 'tekstur', 'polarity': 'positive'}]
1675    [{'term': 'efek minyak', 'polarity': 'negative'}]
415     [{'term': 'tekstur', 'polarity': 'positive'}, ...
Name: aspect_term_sentiment, dtype: object


In [ ]:
# Initialize InstructionsHandler and DatasetLoader
instruct_handler = InstructionsHandler()

# Load instruction set
instruct_handler.load_instruction_set3()  # Load appropriate instruction set

# Create dataset loader
loader = DatasetLoader(train_df_id=id_tr_df, val_df_id=id_val_df)

# Process train dataset
if loader.train_df_id is not None:
    processed_train = loader.create_data_in_aspe_format(
        loader.train_df_id,
        'term',
        'polarity',
        'review',
        'aspect_term_sentiment',
        instruct_handler.aspe['bos_instruct2'],
        instruct_handler.aspe['eos_instruct']
    )

# Process val dataset
if loader.val_df_id is not None:
    processed_val = loader.create_data_in_aspe_format(
        loader.val_df_id,
        'term',
        'polarity',
        'review',
        'aspect_term_sentiment',
        instruct_handler.aspe['bos_instruct2'],
        instruct_handler.aspe['eos_instruct']
    )

In [ ]:
print(loader.train_df_id.columns)
print(loader.val_df_id.columns)

Index(['id', 'kode_produk', 'review', 'aspect_term_sentiment', 'labels',
       'text'],
      dtype='object')
Index(['id', 'kode_produk', 'review', 'aspect_term_sentiment', 'labels',
       'text'],
      dtype='object')


In [ ]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Map:   0%|          | 0/1524 [00:00<?, ? examples/s]

Map:   0%|          | 0/382 [00:00<?, ? examples/s]

In [ ]:
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'kode_produk', 'review', 'aspect_term_sentiment', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1524
    })
    validation: Dataset({
        features: ['id', 'kode_produk', 'review', 'aspect_term_sentiment', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 382
    })
})

In [ ]:
cek_train = pd.DataFrame(id_tokenized_ds["train"][:5])
cek_train

,id,kode_produk,review,aspect_term_sentiment,labels,text,__index_level_0__,input_ids,attention_mask
0,661a16276b6b11825a15b2e6,82166,oke sih produk ini cepat nyerep karena bentuk ...,"[{'polarity': 'positive', 'term': 'tekstur'}]","[3, 15150, 7, 2905, 10, 26093, 1]",Output akan berupa aspek yang meliputi tesktur...,1081,"[3387, 2562, 3, 9, 3304, 3, 1152, 413, 9, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,671ca0654cc623e96df90648,19048,bagus banget! tidak ada whitecast dan gampang ...,"[{'polarity': 'positive', 'term': 'whitecast'}...","[872, 5254, 10, 26093, 6, 3, 15150, 7, 2905, 1...",Output akan berupa aspek yang meliputi tesktur...,155,"[3387, 2562, 3, 9, 3304, 3, 1152, 413, 9, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,65db54c32fdc16dc76afbdbd,101646,"this is sok good, untuk kulit dry dan sensitiv...","[{'polarity': 'positive', 'term': 'efek minyak'}]","[3, 15, 89, 15, 157, 3519, 20856, 10, 26093, 1]",Output akan berupa aspek yang meliputi tesktur...,477,"[3387, 2562, 3, 9, 3304, 3, 1152, 413, 9, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,666dba01e0c470a94a87835d,19048,"sudah lama pakai dan memang sebagus itu, engga...","[{'polarity': 'positive', 'term': 'lainnya'}]","[50, 77, 29, 63, 9, 10, 26093, 1]",Output akan berupa aspek yang meliputi tesktur...,1088,"[3387, 2562, 3, 9, 3304, 3, 1152, 413, 9, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,66696d32c51b55354e62fce8,98204,"sensasi awal pakai ini rasanya dingin, karena ...","[{'polarity': 'negative', 'term': 'lainnya'}]","[50, 77, 29, 63, 9, 10, 31600, 1]",Output akan berupa aspek yang meliputi tesktur...,785,"[3387, 2562, 3, 9, 3304, 3, 1152, 413, 9, 38, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
import optuna
from functools import partial
import os

os.environ["WANDB_DISABLED"] = "true"

use_mps = False

# 1. Definisikan training_args awal tanpa hyperparameter Optuna
training_args = {
    'output_dir': "best_model",
    'evaluation_strategy': "epoch",
    'lr_scheduler_type': 'cosine',
    'warmup_ratio': 0.1,
    'save_strategy': 'no',
    'load_best_model_at_end': False,
    'push_to_hub': False,
    'eval_accumulation_steps': 1,
    'predict_with_generate': True,
    'use_mps_device': use_mps
}

# 2. Definisikan fungsi objektif untuk Optuna
def objective(trial, t5_exp, id_tokenized_ds):
    """ Fungsi yang digunakan Optuna untuk mencari hyperparameter terbaik """

    # Pilih nilai hyperparameter menggunakan Optuna
    learning_rate = trial.suggest_loguniform("learning_rate", 0.00018, 0.0003)
    batch_size = trial.suggest_categorical("per_device_train_batch_size", [8])
    num_train_epochs = trial.suggest_int("num_train_epochs", 5, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 0.02, 0.03)

    # Salin training_args awal dan perbarui dengan hyperparameter Optuna
    current_args = training_args.copy()
    current_args.update({
        'learning_rate': learning_rate,
        'per_device_train_batch_size': batch_size,
        'per_device_eval_batch_size': batch_size * 2,
        'num_train_epochs': num_train_epochs,
        'weight_decay': weight_decay
    })

    # Latih model dengan hyperparameter yang diuji
    model_trainer = t5_exp.train(id_tokenized_ds, **current_args)

    # Evaluasi model menggunakan validation set
    val_pred_labels = t5_exp.get_labels(tokenized_dataset=id_tokenized_ds, sample_set='validation', batch_size=16)
    val_labels = [i.strip() for i in id_ds['validation']['labels']]  # Sesuaikan nama dataset
    _, _, val_f1, _ = t5_exp.get_metrics(val_labels, val_pred_labels)

    return -val_f1

# 3. Jalankan Bayesian Optimization dengan jumlah trial tertentu
n_trials = 100
study = optuna.create_study(direction="minimize")
study.optimize(partial(objective, t5_exp=t5_exp, id_tokenized_ds=id_tokenized_ds), n_trials=n_trials)

# 4. Dapatkan hyperparameter terbaik
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# 5. Update training_args dengan hyperparameter terbaik
training_args.update(best_params)

# 6. Gunakan hyperparameter terbaik untuk pelatihan akhir
final_model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

[I 2025-03-08 07:48:03,788] A new study created in memory with name: no-name-31dae22b-9517-444d-a99d-dfdd4407ed60
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Trainer device: cuda:0

Model training started ....


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.260245
2,No log,0.227271
3,0.304100,0.188775
4,0.304100,0.178599
5,0.304100,0.165495
6,0.125300,0.168405


Model loaded to:  cuda


...

Epoch,Training Loss,Validation Loss
1,No log,0.442672
2,No log,0.386014
3,0.006300,0.433959
4,0.006300,0.390453
5,0.006300,0.389177
6,0.004500,0.393471
7,0.004500,0.400835
8,0.004600,0.404478


Model loaded to:  cuda


100%|██████████| 24/24 [00:13<00:00,  1.78it/s]
[I 2025-03-08 08:54:52,883] Trial 9 finished with value: -0.8094820017559262 and parameters: {'learning_rate': 0.000200252338605389, 'per_device_train_batch_size': 8, 'num_train_epochs': 8, 'weight_decay': 0.020165012255069243}. Best is trial 7 with value: -0.8185801928133216.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Best Hyperparameters: {'learning_rate': 0.00022288231117357943, 'per_device_train_batch_size': 8, 'num_train_epochs': 6, 'weight_decay': 0.02765870774743854}
Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.582508
2,No log,0.524342
3,0.006800,0.344558
4,0.006800,0.403126
5,0.006800,0.420742
6,0.002400,0.423582


In [ ]:
!python -m pip install --upgrade pip
!python -m pip install -U accelerate
!python -m pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1

In [ ]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_val_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'validation', batch_size = 16)
id_val_labels = [i.strip() for i in id_ds['validation']['labels']]

Map:   0%|          | 0/1524 [00:00<?, ? examples/s]

Map:   0%|          | 0/382 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 96/96 [00:54<00:00,  1.77it/s]


Model loaded to:  cuda


100%|██████████| 24/24 [00:13<00:00,  1.77it/s]


In [ ]:
# Evaluasi metrik untuk training set
train_p, train_r, train_f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print("Train Precision: ", train_p)
print("Train Recall: ", train_r)
print("Train F1: ", train_f1)

# Evaluasi metrik untuk testing/validation set
test_p, test_r, test_f1, _ = t5_exp.get_metrics(id_val_labels, id_val_pred_labels)
print("Val Precision: ", test_p)
print("Val Recall: ", test_r)
print("Val F1: ", test_f1)

Train Precision:  0.8476491862567812
Train Recall:  0.8491847826086957
Train F1:  0.8484162895927602
Val Precision:  0.7570422535211268
Val Recall:  0.7543859649122807
Val F1:  0.7557117750439368


In [ ]:
import pandas as pd

# Pastikan index data sesuai dengan dataset asli
id_tr_df = id_tr_df.reset_index(drop=True)
id_val_df = id_val_df.reset_index(drop=True)

# Buat DataFrame hasil prediksi - Training set
train_results = pd.DataFrame({
    'id': id_ds['train']['id'],
    'kode_produk': id_ds['train']['kode_produk'],
    'review': id_ds['train']['review'],
    'Actual Labels (Train)': id_tr_labels,
    'Predicted Labels (Train)': id_tr_pred_labels
})

# Buat DataFrame hasil prediksi - Testing set
val_results = pd.DataFrame({
    'id': id_ds['validation']['id'],
    'kode_produk': id_ds['validation']['kode_produk'],
    'review': id_ds['validation']['review'],
    'Actual Labels (Test)': id_val_labels,
    'Predicted Labels (Test)': id_val_pred_labels
})

# Tampilkan hasil - Training set
print("Training Results")
train_results

Training Results


,id,kode_produk,review,Actual Labels (Train),Predicted Labels (Train)
0,66a5cc1fb7abe21349a93ae0,98204,sampai sudah lupa ini botol ke berapa. praktis...,noaspectterm:none,noaspectterm:none
1,66e13e5e8558eead5050c3e7,101646,"sunscreen favorit, harganya bersahabat dan sec...","harga:positive, tekstur:positive","harga:positive, tekstur:positive"
2,6694963435c88f71c092d6c9,80032,teksturnya cloudy banget dan gampang menyeraap...,tekstur:positive,tekstur:positive
3,6766acb7090d6114e2fcd504,101646,benar benar bagus banget dan cocok banget di a...,"tekstur:positive, whitecast:positive","tekstur:positive, whitecast:positive"
4,669b8fe45f5f3a36cf0f5471,82166,my comfort chemical sunscreen banget pokoknya ...,tekstur:positive,tekstur:positive
...,...,...,...,...,...
319,666f70217ebd2777b684a594,98204,mau coba sunscreen spray dan pilih ini. beli b...,"efek minyak:negative, aroma:negative","efek minyak:negative, aroma:negative"
320,673a046836d5c60310259880,19048,bagus dan sangat recommend digunakan pada kuli...,harga:positive,harga:positive
321,664caea4156dbac34e3b682e,101646,"teksturnya ringan cepat nyerap juga, tapi di m...","tekstur:positive, efek minyak:negative","tekstur:positive, efek minyak:negative"
322,6769fda99f7373f523387347,82166,finally nemu parfume drugstore yang langsung b...,noaspectterm:none,noaspectterm:none


In [ ]:
val_results

,id,kode_produk,review,Actual Labels (Test),Predicted Labels (Test)
0,66a2605c3b7d851c19755544,80032,sudah pakai 1 botol ini dan benar benar sengar...,tekstur:positive,tekstur:positive
1,6693e3b4f6cdc66e59d9d25b,98204,agak susah pakai spray karena enggak tau harus...,noaspectterm:none,noaspectterm:none
2,662e7b2069d737240d8ec30e,101646,produk ini sering banget sold outnya!! tapi ok...,whitecast:neutral,noaspectterm:none
3,6713789d2612647d5fb9b21d,80032,teksturnya tuh agak thick tapi mudah saja diap...,tekstur:neutral,tekstur:negative
4,67669c30667ef73413a7d94f,80032,sunscreennya cocok di kulitku yang oily tekstu...,"efek minyak:negative, tekstur:positive, whitec...","tekstur:positive, whitecast:positive"
...,...,...,...,...,...
76,676e55f7d6b5755272f31b95,80032,untuk dari sisi harganya sunscreen ini sudah o...,"harga:positive, tekstur:negative","harga:positive, tekstur:negative"
77,66a822c036388449cce7297f,80032,menyerapnya lumayan cepat dan bikin lembab jug...,"tekstur:positive, aroma:positive, waterproof:p...",aroma:positive
78,65b0c54d18c248c291bb0dc0,19048,sunscreen cocok untuk oily skin. walaupun sete...,"efek minyak:positive, whitecast:negative","efek minyak:positive, whitecast:positive"
79,66cb0112e266965e71231157,98204,"walaupun baunya menyengat ya, tapi simple to u...",aroma:negative,"aroma:positive, efek minyak:positive"
